In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import cluster, datasets
from matplotlib.lines import Line2D
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

### Conection to spotify

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [3]:
big_df=pd.read_csv("combined_spotify_data.csv")

In [4]:
big_df.rename(columns={'id_x': 'track_id', 'id_y': 'artist_id'}, inplace=True)

In [5]:
big_df


,track_id,track_name,popularity_x,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,artist_id,followers,genres,artist_name,popularity_y
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.4450,...,0.744000,0.1510,0.1270,104.851,3,45tIt06XoI0Iio4LBEVpls,91.0,[],Uli,4.0
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.2630,...,0.000000,0.1480,0.6550,102.009,1,14jtPCOoNZwquk5wd9DxrY,3.0,[],Fernando Pessoa,0.0
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.1770,...,0.021800,0.2120,0.4570,130.418,5,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"['tango', 'vintage tango']",Ignacio Corsini,23.0
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.321,0.0946,...,0.918000,0.1040,0.3970,169.980,3,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"['tango', 'vintage tango']",Ignacio Corsini,23.0
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,...,0.130000,0.3110,0.1960,103.220,4,3BiJGZsyX9sJchTqcSA7Su,11327.0,"['adult standards', 'big band', 'easy listenin...",Dick Haymes,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757165,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],1dy5WNgIKQU6ezkpZs4y8z,2020-10-21,0.765,0.6630,...,0.000297,0.0924,0.6860,150.091,4,1dy5WNgIKQU6ezkpZs4y8z,245944.0,"['alt z', 'alternative r&b', 'bedroom pop', 'i...",ROLE MODEL,67.0
757166,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],37M5pPGs6V1fchFJSgCguX,2020-09-02,0.535,0.3140,...,0.000150,0.0874,0.0663,145.095,4,37M5pPGs6V1fchFJSgCguX,1168213.0,"['alt z', 'electropop', 'indie pop', 'la indie...",FINNEAS,77.0
757167,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']",4jGPdu95icCKVF31CcFKbS,2021-03-05,0.696,0.6150,...,0.000003,0.3050,0.4380,90.029,4,4jGPdu95icCKVF31CcFKbS,45309.0,"['chill r&b', 'indie cafe pop', 'singaporean p...",Gentle Bones,58.0
757168,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']",5ebPSE9YI5aLeZ1Z2gkqjn,2021-03-05,0.696,0.6150,...,0.000003,0.3050,0.4380,90.029,4,5ebPSE9YI5aLeZ1Z2gkqjn,93797.0,"['opm', 'pinoy indie', 'pinoy singer-songwriter']",Clara Benin,53.0


In [6]:
hot_songs_df=pd.read_csv("hot_songs.csv")

In [7]:
hot_songs_df["Song Title"] = hot_songs_df["Song Title"].str.lower()
hot_songs_df["Artist"] = hot_songs_df["Artist"].str.lower()

In [8]:
hot_songs_df=hot_songs_df.drop("Unnamed: 0",axis=1)

In [9]:
hot_songs_df.columns=["title","artist"]

In [10]:
hot_songs_df

,title,artist
0,cruel summer,taylor swift
1,lovin on me,jack harlow
2,paint the town red,doja cat
3,snooze,sza
4,is it over now? (taylor's version) [from the v...,taylor swift
...,...,...
171,bigger than the whole sky,taylor swift
172,good good,"usher, summer walker & 21 savage"
173,vampire,olivia rodrigo
174,strong,anne wilson


# Unsupervised Learning: K-means clustering

In [11]:
big_df=big_df.dropna()

In [12]:
#taking onyl 10% of the data so is not so slow

sampled_df = big_df.sample(frac=0.99)
sampled_df

,track_id,track_name,popularity_x,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,artist_id,followers,genres,artist_name,popularity_y
601260,4mCqxMy4V1B6woxd25vDqg,The Great Divide - Radio Edit,2,247273,0,['Velvetine'],192J7pIS4EJBx0U70kx40D,2013-01-03,0.545,0.7060,...,0.107000,0.0860,0.0941,132.011,4,192J7pIS4EJBx0U70kx40D,3345.0,[],Velvetine,33.0
700441,2CgwcYJKtXdkXYPWc6xClo,"Cello Concerto No.2, Op.126: 1. Largo",7,841533,0,"['Dmitri Shostakovich', 'Mstislav Rostropovich...",0atCvjK2GL6ezQFGOQOYQo,1961-01-01,0.137,0.0315,...,0.842000,0.1060,0.0389,84.239,4,0atCvjK2GL6ezQFGOQOYQo,39589.0,"['classical performance', 'japanese classical ...",Seiji Ozawa,59.0
592388,30UNDv108YstCmbSO5THt5,Intro/Promiscuous,50,242293,0,"['Nelly Furtado', 'Timbaland']",2jw70GZXlAI8QzWeY2bgRc,2006,0.808,0.9700,...,0.000061,0.1540,0.8680,114.328,4,2jw70GZXlAI8QzWeY2bgRc,2509112.0,"['canadian latin', 'canadian pop', 'dance pop'...",Nelly Furtado,78.0
353233,1KmEeR7ppmwYuDOv7ZBPVY,Like A Hero,9,337031,0,['Chyi Yu'],5E94Yc9O9D4N3oTr4SzJLx,1987-12-12,0.444,0.3820,...,0.000000,0.0794,0.2670,80.922,4,5E94Yc9O9D4N3oTr4SzJLx,25781.0,"['c-pop', 'classic mandopop', 'fo jing', 'vint...",Chyi Yu,48.0
470709,2i7U8x3Ed4VE0r8KIAhd0q,Kara Zindan,17,285333,0,['İbrahim Tatlıses'],0gPgE6wLLiPnrakh9WcsdQ,1989-12-07,0.575,0.4310,...,0.000004,0.4480,0.3650,120.769,4,0gPgE6wLLiPnrakh9WcsdQ,800059.0,"['arabesk', 'turkish folk']",İbrahim Tatlıses,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278358,39kzWAiVPpycdMpr745oPj,You Never Know,69,231202,0,['BLACKPINK'],41MozSoPIsD1dJM0CLPjZF,2020-10-02,0.477,0.4350,...,0.000000,0.1200,0.3220,144.144,4,41MozSoPIsD1dJM0CLPjZF,20950428.0,"['k-pop', 'k-pop girl group']",BLACKPINK,87.0
475690,7Dy190FwZJAD1JPc3C4UGI,Prelude: First Snow,35,171400,0,['Michael Gettel'],3PJf9tyCBdYvchFXkl8YLF,2002-01-01,0.370,0.0751,...,0.847000,0.0881,0.0397,141.826,4,3PJf9tyCBdYvchFXkl8YLF,3224.0,['new age'],Michael Gettel,30.0
393647,5v1AyNfoFLxnYVdqNMymIu,Zombie Attack - 2005 Remaster,0,231133,0,['Tankard'],0lKpKsvjBKLUeyVIAPHUy1,2016-07-15,0.268,0.9700,...,0.002170,0.6850,0.0416,88.725,4,0lKpKsvjBKLUeyVIAPHUy1,99604.0,"['crossover thrash', 'death metal', 'german me...",Tankard,39.0
705400,26ss6sCKn9inkL7txKq34p,Coffee Homeground,22,219000,0,['Kate Bush'],1aSxMhuvixZ8h9dK9jIDwL,1978-11-13,0.667,0.3760,...,0.000485,0.2330,0.5140,102.904,4,1aSxMhuvixZ8h9dK9jIDwL,754238.0,"['art pop', 'art rock', 'baroque pop', 'new wa...",Kate Bush,70.0


In [13]:
columns_to_drop = ['track_id', 'track_name', 'explicit', 'artists', 
                   'id_artists', 'release_date', 'artist_id', 
                   'genres', 'artist_name', 'popularity_x', 'popularity_y',"followers"]

In [14]:
sampled_df = sampled_df.drop(columns=columns_to_drop)

In [15]:
sampled_df

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
601260,247273,0.545,0.7060,1,-8.298,0,0.0332,0.005850,0.107000,0.0860,0.0941,132.011,4
700441,841533,0.137,0.0315,0,-26.019,0,0.0410,0.926000,0.842000,0.1060,0.0389,84.239,4
592388,242293,0.808,0.9700,10,-6.098,0,0.0506,0.056900,0.000061,0.1540,0.8680,114.328,4
353233,337031,0.444,0.3820,11,-11.803,1,0.0264,0.602000,0.000000,0.0794,0.2670,80.922,4
470709,285333,0.575,0.4310,1,-9.103,0,0.0421,0.772000,0.000004,0.4480,0.3650,120.769,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278358,231202,0.477,0.4350,5,-6.497,1,0.0523,0.237000,0.000000,0.1200,0.3220,144.144,4
475690,171400,0.370,0.0751,7,-21.149,0,0.0358,0.992000,0.847000,0.0881,0.0397,141.826,4
393647,231133,0.268,0.9700,5,-7.235,1,0.1360,0.000028,0.002170,0.6850,0.0416,88.725,4
705400,219000,0.667,0.3760,6,-9.576,1,0.0391,0.826000,0.000485,0.2330,0.5140,102.904,4


### scale the data 

In [17]:
# Load the saved StandardScaler
with open('transformer.pkl', 'rb') as f:
    loaded_transformer = pickle.load(f)

#transformer = StandardScaler().fit(sampled_df)
cluster_df_normalized = loaded_transformer.transform(sampled_df)

# Now, all features will have the same weight.
cluster_df_normalized = pd.DataFrame(cluster_df_normalized,columns=sampled_df.columns)
cluster_df_normalized

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.076861,-0.053328,0.715030,-1.197519,0.431789,-1.390511,-0.416275,-1.335086,-0.068538,-0.703196,-1.669051,0.493724,0.27752
1,4.119407,-2.385654,-1.874647,-1.482275,-2.747497,-1.390511,-0.374023,1.233319,2.550740,-0.595167,-1.878492,-1.103877,0.27752
2,0.042983,1.450107,1.728633,1.365286,0.826486,-1.390511,-0.322019,-1.192591,-0.449630,-0.335899,1.267303,-0.097634,0.27752
3,0.687453,-0.630693,-0.528936,1.650042,-0.197035,0.719160,-0.453111,0.328941,-0.449848,-0.738845,-1.013029,-1.214805,0.27752
4,0.335770,0.118166,-0.340806,-1.197519,0.287366,-1.390511,-0.368064,0.803461,-0.449834,1.252121,-0.641194,0.117767,0.27752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723630,-0.032465,-0.442049,-0.325448,-0.058495,0.754903,0.719160,-0.312811,-0.689879,-0.449848,-0.519547,-0.804346,0.899479,0.27752
723631,-0.439277,-1.053713,-1.707249,0.511018,-1.873781,-1.390511,-0.402191,1.417545,2.568558,-0.691853,-1.875457,0.821960,0.27752
723632,-0.032934,-1.636794,1.728633,-0.058495,0.622500,0.719160,0.140593,-1.351338,-0.442115,2.532260,-1.868248,-0.953856,0.27752
723633,-0.115471,0.644083,-0.551973,0.226262,0.202506,0.719160,-0.384315,0.954190,-0.448120,0.090814,-0.075854,-0.479678,0.27752


In [18]:
cluster_df_normalized

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.076861,-0.053328,0.715030,-1.197519,0.431789,-1.390511,-0.416275,-1.335086,-0.068538,-0.703196,-1.669051,0.493724,0.27752
1,4.119407,-2.385654,-1.874647,-1.482275,-2.747497,-1.390511,-0.374023,1.233319,2.550740,-0.595167,-1.878492,-1.103877,0.27752
2,0.042983,1.450107,1.728633,1.365286,0.826486,-1.390511,-0.322019,-1.192591,-0.449630,-0.335899,1.267303,-0.097634,0.27752
3,0.687453,-0.630693,-0.528936,1.650042,-0.197035,0.719160,-0.453111,0.328941,-0.449848,-0.738845,-1.013029,-1.214805,0.27752
4,0.335770,0.118166,-0.340806,-1.197519,0.287366,-1.390511,-0.368064,0.803461,-0.449834,1.252121,-0.641194,0.117767,0.27752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723630,-0.032465,-0.442049,-0.325448,-0.058495,0.754903,0.719160,-0.312811,-0.689879,-0.449848,-0.519547,-0.804346,0.899479,0.27752
723631,-0.439277,-1.053713,-1.707249,0.511018,-1.873781,-1.390511,-0.402191,1.417545,2.568558,-0.691853,-1.875457,0.821960,0.27752
723632,-0.032934,-1.636794,1.728633,-0.058495,0.622500,0.719160,0.140593,-1.351338,-0.442115,2.532260,-1.868248,-0.953856,0.27752
723633,-0.115471,0.644083,-0.551973,0.226262,0.202506,0.719160,-0.384315,0.954190,-0.448120,0.090814,-0.075854,-0.479678,0.27752


### Chosing K

In [19]:
# K = range(2, 20)
# inertia = []

# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                     n_init=10,
#                     random_state=1234)
#     kmeans.fit(cluster_df_normalized)
#     inertia.append(kmeans.inertia_)
    
# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Elbow Method showing the optimal k')

In [20]:
K = range(2, 20)
silhouette = []

In [ ]:
# for k in K:
#     kmeans = KMeans(n_clusters=k,
#                     n_init=10,
#                     random_state=1234)
#     kmeans.fit(cluster_df_normalized)
#     silhouette.append(silhouette_score(cluster_df_normalized, kmeans.predict(cluster_df_normalized)))


# plt.figure(figsize=(16,8))
# plt.plot(K, silhouette, 'bx-')
# plt.xlabel('k')
# plt.ylabel('silhouette score')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Slhouette score showing the optimal k')

In [21]:
# Load the saved KMeans model
with open('kmeans.pkl', 'rb') as f:
    loaded_kmeans = pickle.load(f)


In [22]:
# # i would chose 9
# kmeans = KMeans(n_clusters=10, n_init=10, random_state=1234)
# kmeans.fit(cluster_df_normalized)

In [23]:
# # Save the transformer
# with open('transformer.pkl', 'wb') as f:
#     pickle.dump(transformer, f)

# # Save the KMeans model
# with open('kmeans.pkl', 'wb') as f:
#     pickle.dump(kmeans, f)

In [24]:
# Predicting and assigning the clusters:
clusters = loaded_kmeans.predict(cluster_df_normalized)

In [25]:
# Check the size of the clusters (3rd cluster is a bit too much)
pd.Series(clusters).value_counts().sort_index()

0      5732
1     10299
2      7969
3     10606
4      8362
      ...  
95     8125
96     3372
97     4410
98     6647
99     8847
Length: 100, dtype: int64

In [28]:
loaded_kmeans.inertia_ 

2551990.3903765795

In [29]:
clustered_df=sampled_df.assign(clusters = clusters)
clustered_df

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,clusters
601260,247273,0.545,0.7060,1,-8.298,0,0.0332,0.005850,0.107000,0.0860,0.0941,132.011,4,55
700441,841533,0.137,0.0315,0,-26.019,0,0.0410,0.926000,0.842000,0.1060,0.0389,84.239,4,73
592388,242293,0.808,0.9700,10,-6.098,0,0.0506,0.056900,0.000061,0.1540,0.8680,114.328,4,56
353233,337031,0.444,0.3820,11,-11.803,1,0.0264,0.602000,0.000000,0.0794,0.2670,80.922,4,30
470709,285333,0.575,0.4310,1,-9.103,0,0.0421,0.772000,0.000004,0.4480,0.3650,120.769,4,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278358,231202,0.477,0.4350,5,-6.497,1,0.0523,0.237000,0.000000,0.1200,0.3220,144.144,4,74
475690,171400,0.370,0.0751,7,-21.149,0,0.0358,0.992000,0.847000,0.0881,0.0397,141.826,4,8
393647,231133,0.268,0.9700,5,-7.235,1,0.1360,0.000028,0.002170,0.6850,0.0416,88.725,4,13
705400,219000,0.667,0.3760,6,-9.576,1,0.0391,0.826000,0.000485,0.2330,0.5140,102.904,4,38


# Song recomender

In [30]:
track = input("Enter a song that you like: ")


if hot_songs_df['title'].str.lower().str.contains(str(track).lower()).any() == True:
        print('here what I found for you! ')
        print(hot_songs_df['title'].sample())
else:
    # get the track name
    results = sp.search(q="track:" + track, type="track")
    
    # find the song uri with spotipy
    song_uri = results['tracks']['items'][0]['uri'].split(':')[2]
    
    #find song features
    song_features = sp.audio_features(song_uri)
    
    #clean song features
    new = pd.DataFrame.from_dict(song_features)
    y_pred = new.drop(columns=['type', 'id' , 'uri', 'track_href', 'analysis_url'])
    
    
    
    # scale the data
    y_scaled = transformer.transform(y_pred)
    y_scaled = pd.DataFrame(y_scaled,columns=y_pred.columns)
    
    # use the Kmeans created before to predict the cluster for the song
    look_for = kmeans.predict(y_scaled)
    print('Here what I found for you !!!')
    
    # choose a random row from the platlist with same cluster
    print(playlist_clustered.loc[playlist_clustered.clusters == look_for[0],['title','artist','url']].sample(1))

Enter a song that you like: jam
here what I found for you! 
61    the painter
Name: title, dtype: object
